In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

%pylab inline
sns.set(rc={'figure.figsize': (9, 5)})


import warnings
warnings.filterwarnings("ignore")
np.random.seed(0)

Populating the interactive namespace from numpy and matplotlib


In [2]:
def PrintTriangular(mas, i, lines=None):
    if lines is None:
        lines = len(mas)
        
    for line in range(lines):
        for n in range(line + 1 - i):
            print('{0:7.4f}'.format(mas[line][n]), end=' ')
        print()
        
def RicherdsonExtrapolation(func, a, b, EPS=10, p=2, q=1, r=2, N=2, S=25):
    U = np.zeros((S,S))
    R = np.zeros((S,S))
    p_eff = np.zeros((S,S))

    s = 1
    U[0][0] = getSpectrum(func, a, b, N)
    while True:
        U[s][0] = getSpectrum(func, a, b, r**s*N)

        for n in range(s):
            R[s][n] = (U[s][n] - U[s-1][n]) / (r**(p + n*q) - 1)
            
            p_eff[s][n] = log(abs(R[s-1][n]/R[s][n])) / log(r)
            
            U[s][n + 1] = U[s][n] + R[s][n]
            
            if abs(R[s][n]) < (0.1)**EPS:
                return (U[s][n], R[s][n], p_eff[s][n], p*(s-1)+n*q | p, s, n, U, R, p_eff)

            
        
        s += 1
        if s > 23: 
            return (U[s-1][s-1], R[s-1][s-1], p_eff[s-1][s-3], p + s*q, s, s, U, R, p_eff)
        
def show_answer(ans):
    re = " \
Ответ: {0:12.10f}\n \
Погрешность: {1:12.10f}\n \
Эффективный порядок точности: {2:7.10f}\n \
Теоретический порядок точности: {3:7.5f}\n \
    ".format(ans[0], ans[1], ans[2], ans[3])
    print(re)
    PrintTriangular(ans[-3], 0, ans[4])
    PrintTriangular(ans[-2], 1, ans[4])
    PrintTriangular(ans[-1], 2, ans[4])

#### Поиск минимального собсвенного значения задачи Штурма-Лиувилля при помощи алгоритма обратной итерации с использованием методики Ричердсона для задачи:
$$
\begin{equation*}
 \begin{cases}
   U_{xx} + 4U_x + \lambda U = 0, 
   \\
   U(0) = 0, U(1) = 0, 
   \\
   x \in [0, 1]
 \end{cases}
\end{equation*}
$$

1. Данной задаче будет соответсвовать матрица A в бесконечномерном пространсве
2. Если использовать в качестве производных их конечно разностную апроксимацию со вторым порядком точности, то коэффициенты данной матрицы будут выражаеться по формулам:
   - $a_{n,n-1} = -\frac{1}{h} - \frac{a}{2h}$, $n = 2,..,N$
   - $a_{n,n} = \frac{2}{h^2}$, $n = 1,..,N$
   - $a_{n,n+1} = \frac{1}{h} + \frac{a}{2h}$, $n = 1,..,N$
   
где a, в данном смучая, ровняеться 4, h - шаг равномерной сетки по х

3. Таким образом, получим систему уравнений $Ay = \lambda y$, для которой можно применить метод обратной итерации, тогда 
   - $ \frac{(x^{(s+1)},x^{(s+1)})}{(x^{(s)},x^{(s+1)})} = \lambda_n \rightarrow  \lambda_{min}$, при $s \rightarrow +\infty$

In [3]:
def getSpectrum(getA, a, b, N):
    A = getA(a, b, N)
    X = np.random.rand(N)*10
    X1 = X
    for _ in range(10):
        X = X1
        X1 = A.dot(X)
    
    lambda_ = np.sum(X1 * X1) / np.sum(X1 * X)
        
    return lambda_

In [4]:
def task1(a, b, N):
    h = (b - a) / N
    
    B = np.diagflat([-1/h - 2/h]*(N-1), k=1)
    B += np.diagflat([2/h**2]*N)
    B += np.diagflat([1/h + 2/h]*(N-1), k=-1)
    
    A = np.linalg.inv(B)    
    return A

In [7]:
ans = RicherdsonExtrapolation(task1, a=0, b=1, EPS=15)
show_answer(ans)

 Ответ: 0.0000000000
 Погрешность: 0.0000000000
 Эффективный порядок точности:    -inf
 Теоретический порядок точности: 3.00000
     
 0.1250 
 0.0312 -0.0000 

-0.0313 


